 # PySpark笔记之一：读写并处理数据

## Overview
PySpark是提供了Python语言API接口的Spark，经过我的初步使用，未发现和Scala API的Spark有太大差别。且我们服务器上已经配置好了PySpark，正好配合Jupyter notebook使用来进行机器学习离线训练模型。
从这篇文章开始，将从一个算法工程师的视角去记录一下Spark的使用。
Spark在我看来就是一个计算工具，用来处理单机计算不了的问题。单机解决不了的问题，无非就是数据太大，内存，磁盘以及处理器不够用，然后把数据拆开给多台机器一起计算。Spark相比较于Hadoop，有一个长处就在于计算。而在机器学习过程当中，从数据预处理到模型训练都需要运算，尤其是数据量比较大的时候。我们尝试对比单机机器学习和分布式机器学习的方式，来加强对Spark的理解。

### 1. 建立Spark会话并读取文件

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
 
spark = SparkSession.builder.appName("read data")\
    .master("local") \
    .config("spark.cores.max", "10") \
    .config("spark.driver.memory", "2G") \
    .config("spark.executor.memory", "2G")  \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

In [2]:
spark

我们在单机上是这样读数据的（一般是从csv文件中读取：

In [3]:
import pandas as pd
df_single = pd.read_csv('/home/robin/datatsets/train.csv')

Pandas会把csv文件读取成DataFrame。
而Spark的处理逻辑也是一样的，会将数据从csv/json/hive/数据库/Hadoop读取为DataFrame，让分析师和算法工程师无缝对接。而语法也是比较简单。

In [4]:
df_spark = spark.read.format("csv").option("header", "true").load('/home/robin/datatsets/train.csv')

In [5]:
df_spark

DataFrame[PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [6]:
df_spark.dtypes

[('PassengerId', 'string'),
 ('Survived', 'string'),
 ('Pclass', 'string'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'string'),
 ('SibSp', 'string'),
 ('Parch', 'string'),
 ('Ticket', 'string'),
 ('Fare', 'string'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

我们可以看到，每一个字段的数据类型都被识别为string，这是因为我们load文件时，没有加inferSchema=True。加上这个条件就正常了。

In [7]:
df_spark = spark.read.csv(path='/home/robin/datatsets/train.csv', sep=',',  header=True, inferSchema=True)
df_spark.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

### 2. 划分数据集和筛选特征

In [8]:
df_train = df_spark.filter(df_spark.Sex == 'male')
df_train.count()
df_val = df_spark.filter(df_spark.Sex == 'female') 
df_val.count()

314

填充空值：

In [9]:
df_train = df_train.fillna(0.0)
df_val = df_val.fillna(0.0)

### 3. 保存处理好的数据

In [10]:
df_train.toPandas().to_csv("训练集特征.csv", index=False)
df_val.toPandas().to_csv("验证集特征.csv", index=False)

### 4. 总结

In [11]:
总体来说，Spark在读取数据的时候，和Pandas的体验是很接近的。可以认为是一个分布式版本的Pandas。

SyntaxError: invalid character in identifier (<ipython-input-11-d9d3afaf4003>, line 1)